[Amirhossein Mahmoudi](https://ammahmoudi.github.io)
# Adams–Bashforth Method with Iterative Correctors
solution to [Numerical Analysis (R. Burden, Aires, A. Burden) [10th Ed.]](https://www.amazon.com/Richard-Burden-Numerical-Analysis-Hardcover/dp/B00SB3UL20) Section 5.6 question 15

![5_6_15](images/5_6_15.png)

In [6]:
import pandas as pd
from math import *
import numpy as np

# Corrector Values for AB Method 2,3,4,5 step

In [7]:
correctors = [
	[2, 3, -1],
	[12, 23, -16, 5],
	[24, 55, -59, 37, -9],
	[720, 1901, -2774, 2616, -1274, 251]
]

# RK-4 Method

In [8]:
def rk4(f, t, y,h):
	k1 = h*f(t, y)
	k2 = h*f(t + h/2, y + k1/2)
	k3 = h*f(t + h/2, y + k2/2)
	k4 = h*f(t + h, y + k3)
	return y + (k1 + 2*k2 + 2*k3 + k4)/6

# Adams–Bashforth method with rk4 initializer

In [9]:
def abN_rk4(f, f_actual, t, y, p, h, N,correctors,corrector_iterations,correction):
	#coeffients
	corrector = correctors[N-2]
	#number of steps
	n = int((p-t)/h)
	#intialize time array
	time = [t+h*i for i in range(n+1)]
	#initialze y_vaules
	y_values = [y]
	#make inital values from rk4
	for i in range(1,N):
		y_values.append(rk4(f, time[i-1], y_values[i-1],h))
	#run the algorithm
	for i in range(N, n+1):
		fix = 0
		for j in range(N):
			fix += corrector[j+1] * f(time[i-j-1], y_values[i-j-1])
		yn = y_values[i-1] + h/corrector[0]*fix
	
		if N == 4 and correction:
			for ci in range(corrector_iterations):
				fix = 9*f(time[i], yn)
				coeffs = [19, -5, 1]
				for j in range(len(coeffs)):
					fix += coeffs[j] * f(time[i-j-1], y_values[i-j-1])
				yn = y_values[i-1] + h/24*fix

		y_values.append(yn)
	y_actual=np.ones_like(time)
	for i in range(len(y_values)):
		y_actual[i]=f_actual(time[i])

	return np.c_[time,y_values,y_actual]


# Test

In [10]:
f = lambda t, y: 1+y/t+(y/t)**2

f_actual = lambda t: t*tan(log(t))

t = 1
y = 0
p = 3
h = 0.2

result=pd.DataFrame()
for i in range(2,5):
    result_n=abN_rk4(f, f_actual, t, y, p, h, 4,correctors,i,correction=True)
    if i==2:
        result=pd.DataFrame(result_n[:,:2],columns=['time','2-iteration'])
    else:
        result[str(i)+"-iteration"]=pd.Series(result_n[:,1])
result['exact']=pd.Series(result_n[:,2])
result.head(n=12)

    
    

,time,2-iteration,3-iteration,4-iteration,exact
0,1.0,0.000000,0.000000,0.000000,0.000000
1,1.2,0.221246,0.221246,0.221246,0.221243
2,1.4,0.489684,0.489684,0.489684,0.489682
3,1.6,0.812752,0.812752,0.812752,0.812753
4,1.8,1.199462,1.199465,1.199466,1.199439
5,2.0,1.661343,1.661351,1.661352,1.661282
6,2.2,2.213626,2.213641,2.213643,2.213502
7,2.4,2.876784,2.876811,2.876814,2.876551
8,2.6,3.678894,3.678943,3.678948,3.678475
9,2.8,4.659412,4.659500,4.659509,4.658665


In [16]:
def MSE(y_true,y_pred):
    return np.sum((y_true-y_pred)**2)
print(MSE(result['2-iteration'].values,result['exact'].values))

2.6077517752537257e-06


In [26]:
errors=[]
for c in result.columns:
    if(c!='time' and c!='exact'):
        errors.append(MSE(result[c].values,result['exact'].values))
errors

[2.6077517752537257e-06, 3.271533494902964e-06, 3.3484177610901586e-06]